In [1]:
# Used python libraries

import cv2
import os
import fnmatch
import re
import trackpy as tp 
import numpy 
import matplotlib.pylab as plt 
import matplotlib
import pims_nd2
import numpy as np
from matplotlib.widgets import TextBox
import pandas as pd

from cv2_rolling_ball import subtract_background_rolling_ball


In [10]:
# Folder with the .nd2 file

folder = r'C:\Users\Julia\Downloads'

# Имя файла

filename = 'ulyana_002'

# Microscopy channels:
    
# If there are 3 channels: 405 (Хёкст), 488 (DiOC6, CD61) and 640 (Annexin-V-Alexa647, P-Selectin-Alexa647)
# 405 - 0
# 488 - 1
# 640 - 2

# If there are 2 channels 488 (DiOC6, CD61) and 640 (Annexin-V-Alexa647, P-Selectin-Alexa647)
# 488 - 0
# 640 - 1


# Default
channel = 0

In [11]:
class RemoveBackground: 
    def __init__(self, trace, channel_number=0):

        self.trace = trace
        frames = pims_nd2.ND2_Reader(self.trace)
        
        self.frames = frames
        self.x = self.frames.sizes.get('x')
        self.y = frames.sizes.get('y')
        self.channel_number = channel_number
        self.remove_bg_number = 0
        self.c = frames.sizes.get('c')
        self.z = frames.sizes.get('m')
        
        self.fig = plt.figure()
        
        plt.axis('off')
        
        
        
        grid_sz = 15
        max 
        plt.subplot2grid((grid_sz,grid_sz), (0,0), colspan=grid_sz, rowspan=grid_sz-1)
        self.img=self.frames[0]
        self.zerozero = plt.imshow(self.img)
        button_ax = plt.subplot2grid((grid_sz,grid_sz ), (grid_sz-1,grid_sz*3//6))
        z_stack_ax = plt.subplot2grid((grid_sz,grid_sz ), (grid_sz-1,grid_sz*1//6))
        channel_ax = plt.subplot2grid((grid_sz,grid_sz ), (grid_sz-1,grid_sz*5//6))
        channel_error=0
        
        self.zerozero.axes.get_xaxis().set_visible(False)
        self.zerozero.axes.get_yaxis().set_visible(False)
        self.done_button = plt.Button(button_ax, 'Done')
        self.done_button.on_clicked(self.on_button_press)
        self.ChannelBox = TextBox(channel_ax, 'Rolling ball size')
        self.ChannelBox.on_submit(self.on_remove_bg_number_update_process)
        self.CellBox = TextBox(z_stack_ax, 'Zstack number')
        self.CellBox.on_submit(self.on_z_stack_number_update_process)
        plt.show()
        self.fig.canvas.start_event_loop(timeout=-1)
        
    def on_button_press(self, event):
            self.pixel_size = self.frames[0].metadata['mpp']
            self.frames.close()
            self.img = self.img[:]
            #print ("X dimension = ", x, "Y dimension = " , y, "Time length = ", t)

            plt.close()
            self.fig.canvas.stop_event_loop()
            
    def on_z_stack_number_update_process(self, val):
            try:
                self.zstack_number = int(eval(val))
                self.frames.default_coords['m'] = self.zstack_number
                self.img=self.frames[0]
                 
                self.zerozero.set_data(self.img)
            except:  
                print ('Error!')
    def on_remove_bg_number_update_process(self, val):
            try:
                self.remove_bg_number = int(eval(val))
                
                image_c, background = subtract_background_rolling_ball(np.uint8(self.frames[0]), eval(val), light_background=False,
                                                     use_paraboloid=False, do_presmooth=False)
                print(np.mean(image_c))
                #self.frames.default_coords['c'] = self.channel_number
                self.img = image_c
                self.zerozero.set_data(self.img)
            except Exception as e:  
                print ('Error!',e)
                

In [12]:
%matplotlib qt

class ImageProcessingGUI():
    def __init__(self, img, img_labels, trajectories, dictionary_frames):

        self.fig = plt.figure(figsize=(20,15))
        self.img = img
        self.img_labels = img_labels
        grid_sz = 15
        self.currentvideoindice = 1
        self.dictionary_frames = dictionary_frames
        self.currtype = 'not specified'
        self.trajectories = trajectories
        self.curr_type_label = ''
        plt.subplot2grid((grid_sz, grid_sz), (0, 0), colspan=grid_sz, rowspan=grid_sz - 1)

        tp.plot_traj(self.trajectories, label=True)
        self.zerozero = plt.imshow(self.img[self.currentvideoindice], alpha=0.9)
        # self.imgprev = plt.imshow(self.img_labels[self.currentvideoindice], alpha = 0.6, cmap = 'jet')
        # plt.plot(trajectories)

        z_stack_ax = plt.subplot2grid((grid_sz, grid_sz), (grid_sz - 1, grid_sz * 5 // 6))
        z_stack_ax1 = plt.subplot2grid((grid_sz, grid_sz), (grid_sz - 1, grid_sz * 7 // 9))
        self.selected_cell_labels = []
        self.cell_labels = img_labels
        self.cell_mask = None

        self.CellBox = TextBox(z_stack_ax, 'Leuco Traj N')
        self.CellBoxType = TextBox(z_stack_ax1, 'Leuco Type')
        self.CellBox.on_submit(self.on_z_stack_number_update_process)
        
        self.CellBoxType.on_submit(self.on_type_number_update_process)
        self.zerozero.axes.get_xaxis().set_visible(False)
        self.zerozero.axes.get_yaxis().set_visible(False)
        plt.title('Use sliders to choose current slide')
        plt.axis('off')
        # self.cid = self.fig.canvas.mpl_connect('button_press_event', self.on_mouse_click)
        slider_ax1 = plt.subplot2grid((grid_sz, grid_sz), (grid_sz - 1, grid_sz * 1 // 6))
        button_ax = plt.subplot2grid((grid_sz, grid_sz), (grid_sz - 1, grid_sz * 2 // 6))

        self.done_button = plt.Button(button_ax, 'Done')
        self.done_button.on_clicked(self.on_button_press)
        self.cutting = plt.Slider(slider_ax1, 'Current slide', 0, (np.shape(img_labels)[0] - 1),
                                  valinit=self.currentvideoindice, valstep=1)
        self.cutting.on_changed(self.on_slider_process)
        self.enum_particles = 0
        plt.show()
        self.fig.canvas.start_event_loop(timeout=-1)

    def on_button_press(self, event):
        plt.close()
        self.fig.canvas.stop_event_loop()

    def on_slider_process(self, val):
        self.currentvideoindice = int(val)
        self.zerozero.set_data(self.img[self.currentvideoindice])
        self.zerozero.set_norm(matplotlib.colors.Normalize())
        self.zerozero.set_cmap('viridis')
        
        # self.imgprev.set_data(self.img_labels[self.currentvideoindice])
        plt.draw()

    def on_z_stack_number_update_process(self, val):
        try:
            self.label = np.array(eval(val),ndmin=1)
           
            print(self.label)
            self.enum_particles+= 1
            print({'indices':self.label,'celltype':self.currtype} )
            self.CellBox.set_val('')
            self.selected_cell_labels.append({'indices':self.label,'celltype':self.currtype} )
            
        except Exception as e:
            print('Error1!', e)
            
    def on_type_number_update_process(self, val):
        try:
            self.currtype =  val
            #self.selected_cell_labels.append(self.label)
        except Exception as e:
            print('Error2!',e)

def get_z_size(trace):
    frames = pims_nd2.ND2_Reader(trace)
    z = frames.sizes.get('m')

    pixel_size = frames[0].metadata['mpp']
    print(frames.metadata)
    frames.close()
    return z, pixel_size

def GetFilmingInitTime(trace):
    
    frames = pims_nd2.ND2_Reader(trace)
    datetime_init = (frames.metadata['time_start'])
    
    frames.close()
    return datetime_init




def getND2timeframes(z_stack, trace,channel_number=1):
    frames = pims_nd2.ND2_Reader(trace)
    
    if (z_stack):
        frames.default_coords['m'] = z_stack
    frames.default_coords['c'] = channel_number
    
    t = frames.sizes.get('t')
    times = numpy.zeros((t))
    
    for i in range(t):
        times[i] = frames[i].metadata['t_ms'] / 1000
    return times    
def getNd2efunc(z_stack, trace,ret_rolling_ball, channel_number=1, max_chunx = None, split_image = None):
    frames = pims_nd2.ND2_Reader(trace)
    x = frames.sizes.get('x')
    y = frames.sizes.get('y')
    
    if (z_stack):
        frames.default_coords['m'] = z_stack
    frames.default_coords['c'] = channel_number
    
    t = frames.sizes.get('t')
    times = numpy.zeros((t))
    if (split_image!=None):
        number = (np.sqrt(max_chunx))+1
        one_size_x = int(np.floor(x/number))
        nx = int(split_image%number)
        ny = int(split_image//number)
        one_size_y = int(np.floor(y/number))
        print(nx,ny,one_size_x,one_size_y)
        
        img = numpy.zeros((t, one_size_x, one_size_y))
    else:
        img = numpy.zeros((t, x, y))
    #print(x,y,nx,ny,(nx)*one_size_x,(ny)*one_size_y)    
    for i in range(t):
        
        #frames_subtract, background = subtract_background_rolling_ball(frames[i], ret_rolling_ball, light_background=False,
                                     #use_paraboloid=False, do_presmooth=True)
        #img[i, :, :] = frames_subtract
        if (split_image!=None):
            img[i, :, :] = frames[i].T[nx*one_size_x:(nx+1)*one_size_x,ny*one_size_y:(ny+1)*one_size_y]
        else:
            img[i, :, :] = frames[i].T
            
        times[i] = frames[i].metadata['t_ms'] / 1000    
        
    frames_fon = frames[:]
    frames.close()
    frames = img[:]

    frames_fon = frames.copy()
    f = tp.batch(frames_fon[:], 91)
    t = tp.link_df(f, int(np.floor(15*2.875)), memory=3)
    f = tp.filter_stubs(t, 4)
    """
        img = img[:10]
        times = times[:10]"""
    dictionary = {}

    for i in (f['particle']):
        particle_center_list = []
        key = (f[f['particle'] == i])
        frame_list_for_particles = (f[f['particle'] == i])['frame']

        for j in frame_list_for_particles:
            centurx1 = key[key['frame'] == j]
            centurx = np.float64(centurx1['x'])
            century = np.float64(centurx1['y'])
            size = np.float64(centurx1['size'])
            particle_center_list.append([j, centurx, century, size])

        dictionary[i] = particle_center_list
    frame_labels = np.zeros_like(frames)
    # print (dictionary)

    binarize_video = np.zeros_like(frames_fon)

    for i in range(np.shape(binarize_video)[0]):
        ret, binarize_video[i] = cv2.threshold(np.uint8(frames_fon[i] * 255 / np.max(frames_fon[i])), 0, 255,
                                               cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    if (split_image!=None):    
        return binarize_video, frames_fon, frame_labels, f, dictionary, times, split_image
    else:
        return binarize_video, frames_fon, frame_labels, f, dictionary, times, z_stack

In [13]:
ret_rolling_ball = 300

from cv2_rolling_ball import subtract_background_rolling_ball

TRACE = folder + '/' + filename
z_max, mpp = get_z_size(TRACE + '.nd2')

bg = RemoveBackground(TRACE + '.nd2')
ret_rolling_ball  = bg.remove_bg_number


C:\ProgramData\Anaconda3\lib\site-packages\pims\base_frames.py:522: UserWarning: Please call FramesSequenceND.__init__() at the start of thethe reader initialization.
  warn("Please call FramesSequenceND.__init__() at the start of the"


{'width': 512, 'width_bytes': 2048, 'height': 512, 'components': 2, 'bitsize_memory': 16, 'bitsize_significant': 14, 'sequence_count': 760, 'tile_width': 512, 'tile_height': 512, 'compression': None, 'compression_quality': 4294967197, 'plane_count': 2, 'angle': 0.0, 'calibration_um': 0.159555420061708, 'time_start_jdn': 2458792.117578947, 'time_start': datetime.datetime(2019, 11, 4, 17, 49, 18, 821020), 'time_start_utc': datetime.datetime(2019, 11, 4, 14, 49, 18, 821020), 'objective': 'Apo TIRF 100x Oil DIC N2', 'magnification': -1.0, 'NA': 1.49, 'refractive_index1': 1.515, 'refractive_index2': 1.515, 'pinhole': 0.0, 'zoom': 1.0, 'projective_mag': -1.0, 'image_type': 'normal', 'z_home': None, 'plane_0': {'components': 1, 'rgb_value': (0.0, 0.9568627450980393, 1.0), 'name': '488 nm', 'oc': 'Stacey_flow', 'emission_nm': 488.0}, 'plane_1': {'components': 1, 'rgb_value': (1.0, 0.0, 0.0), 'name': '640 nm', 'oc': 'Stacey_flow', 'emission_nm': 640.0}}


In [14]:
from joblib import Parallel, delayed

max_chunks = 9
array_of_results = np.zeros(z_max)
#ret_rolling_ball = 0
# [frames, times, mpp] = getNd2efunc(trace=TRACE+'.nd2', channel_number = 1, zstack = z_max)

#getNd2efunc(0, TRACE + '.nd2', ret_rolling_ball, channel,max_chunks, 1)
if (z_max):
    all_z_stack_results = (Parallel(n_jobs=-1)(delayed(getNd2efunc)(i, TRACE + '.nd2', ret_rolling_ball, channel) for i in range(z_max)))
else:
    all_z_stack_results = (Parallel(n_jobs=-1)(delayed(getNd2efunc)(0, TRACE + '.nd2', ret_rolling_ball, channel,max_chunks, i) for i in range(max_chunks)))
zstack_num = z_max

In [20]:
for all_elems in all_z_stack_results:
    all_coords_array_t = []
    all_coords_array_x = []
    all_coords_array_y = []
    all_coords_array_names = []
    percent = []
    binarize_video, frames_fon, frame_labels, f, dictionary, times, zstack_current_num = all_elems
    gui = ImageProcessingGUI(frames_fon, frame_labels, f, dictionary)
    label_list = gui.selected_cell_labels
    
    all_velocities_array = []
    print(gui.selected_cell_labels)
    for all_neutrophils in gui.selected_cell_labels:
        velocity_array = []
        x_array = []
        y_array = []
        t_array = []
        name_array = []
        for j in (all_neutrophils['indices']):
            #j = int(re.sub("[^0-9]", "", j))
            #velocities = np.zeros(len(dictionary[j]))
            for frame_enum in range(1, len(dictionary[j])):
                frame_nr = dictionary[j][frame_enum][0]
                delta_t = times[frame_nr] - times[dictionary[j][frame_enum - 1][0]]
                delta_x = dictionary[j][frame_enum][1] - dictionary[j][frame_enum - 1][1]
                delta_y = dictionary[j][frame_enum][2] - dictionary[j][frame_enum - 1][2]
    
                velocity = np.sqrt(delta_x ** 2 + delta_y ** 2)
                velocity = velocity * mpp / delta_t
                velocity_array.append(velocity)
                t_array.append(times[frame_nr])
                x_array.append(dictionary[j][frame_enum][1])
                y_array.append(dictionary[j][frame_enum][2])
                mask = np.zeros_like(binarize_video[frame_nr])
                size_leuco = dictionary[j][frame_enum][3]
                x_this_slide = dictionary[j][frame_enum][1]
                y_this_slide = dictionary[j][frame_enum][2]
                mask[int(y_this_slide - size_leuco * 1.2):int(y_this_slide + size_leuco * 1.2),
                int(x_this_slide - size_leuco * 1.2):int(x_this_slide + size_leuco * 1.2)] += 1
                binarize_video[frame_nr][mask == 1] *= 0
                name_array.append(all_neutrophils['celltype'])
        all_velocities_array.append(velocity_array)
        all_coords_array_t.append(t_array)
        all_coords_array_x.append(x_array)
        all_coords_array_y.append(y_array)
        all_coords_array_names.append(name_array)

    for j in range(np.shape(binarize_video)[0]):
            frame_thrombus_per_cent = len(binarize_video[j][binarize_video[j] != 0]) / (
                        np.shape(binarize_video)[2] * np.shape(binarize_video)[1])
            percent.append(frame_thrombus_per_cent)
    fig = plt.figure() 
    
    time = times.tolist()
    first = time.index(min(time, key=lambda x: abs(x - 300)))
    fig.suptitle('300 second clot, field #'+str(zstack_current_num+1))
    plt.imshow(frames_fon[first])        
    #plt.imshow(binarize_video[first], alpha = 0.3)
    #plt.show()
    fig = plt.figure()
    fig.suptitle('600 second clot, field #'+str(zstack_current_num+1))
    plt.imshow(frames_fon[-1]) 
    #plt.imshow(binarize_video[-1], alpha = 0.3)
    #plt.show()

    import xlwt

    ex = xlwt.Workbook()
    vel = ex.add_sheet('Velocity', cell_overwrite_ok=True)
    thrombus = ex.add_sheet('Clot', cell_overwrite_ok=True)
    Mean_vals = ex.add_sheet('Mean values', cell_overwrite_ok=True)
    coords = ex.add_sheet('Coords', cell_overwrite_ok=True)
    num = 0
    for different_velocities in (all_velocities_array):
        num += 1
        Mean_vals.write(1,2 * num+4, np.mean(different_velocities))
        Mean_vals.write(0,2*num+4,"Mean Velocity №"+str(num))
        vel.write(0,2*num + 0, 'Velocity №'+str(num))
        for i in range(len(different_velocities)):
            vel.write(i+1, 2*num + 0, different_velocities[i])
    num = 0        
    for different_coords in (all_coords_array_x):
        num += 1
        coords.write(0,4 * num + 0, 'Coords X №'+str(num))
        for i in range(len(different_coords)):
            coords.write(i+1, 4 * num + 0, different_coords[i])
    num = 0        
    for different_coords_y in (all_coords_array_y):
        num += 1
        coords.write(0,4 * num + 1, 'Coords Y №'+str(num))
        for i in range(len(different_coords_y)):
            coords.write(i+1, 4 * num + 1, different_coords_y[i])
    num = 0        
    for different_coords_t in (all_coords_array_t):
        num += 1
        coords.write(0,4 * num +2, 'T №'+str(num))
        for i in range(len(different_coords_t)):
            coords.write(i+1, 4 * num +2, different_coords_t[i])
    num = 0        
    for different_names in (all_coords_array_names):
        num += 1
        coords.write(0,4 * num +3,  'Type №'+str(num))
        vel.write(0,2 * num +1,  'Type №'+str(num))
        Mean_vals.write(0,2 * num+5, 'Type №'+str(num))
        Mean_vals.write(1,2 * num+5, different_names[1])
        for i in range(len(different_names)):
            
            coords.write(i+1, 4 * num + 3, different_names[i])
            vel.write(i+1, 2 * num + 1, different_names[i])
    all_names_for_means = np.unique(all_coords_array_names)          
    num = 0  
    thrombus.write(0, 2, 'Clot square')
    thrombus.write(0, 0, 'Time')
    for j in range(len(percent)):
        thrombus.write(j+1, 2, percent[j])
        thrombus.write(j+1, 0, time[j])

    first = time.index(min(time, key=lambda x: abs(x - 300)))
    print(percent[first] * 100, '\t', percent[-1] * 100)
    Mean_vals.write(1, 0, '300s')
    Mean_vals.write(1, 1, '600s')
    Mean_vals.write(0, 3, 'Mean velocities')
    Mean_vals.write(2, 0, percent[first])
    Mean_vals.write(2, 1, percent[-1])

    ex.save(TRACE + '_field' + str(zstack_current_num + 1) + '.xls')

    plt.show()

[6]
{'indices': array([6]), 'celltype': 'not specified'}
Error1! unexpected EOF while parsing (<string>, line 0)
[2]
{'indices': array([2]), 'celltype': 'not specified'}
Error1! unexpected EOF while parsing (<string>, line 0)
Error1! unexpected EOF while parsing (<string>, line 0)
[{'indices': array([6]), 'celltype': 'not specified'}, {'indices': array([2]), 'celltype': 'not specified'}]


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


9.337615966796875 	 10.486602783203125
[]
8.946990966796875 	 9.383773803710938
[13]
{'indices': array([13]), 'celltype': 'not specified'}
Error1! unexpected EOF while parsing (<string>, line 0)
Error1! unexpected EOF while parsing (<string>, line 0)
[{'indices': array([13]), 'celltype': 'not specified'}]
10.6414794921875 	 10.388565063476562
[6]
{'indices': array([6]), 'celltype': 'not specified'}
Error1! unexpected EOF while parsing (<string>, line 0)
[14]
{'indices': array([14]), 'celltype': 'not specified'}
Error1! unexpected EOF while parsing (<string>, line 0)
Error1! unexpected EOF while parsing (<string>, line 0)
[{'indices': array([6]), 'celltype': 'not specified'}, {'indices': array([14]), 'celltype': 'not specified'}]


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


8.155059814453125 	 9.90447998046875
[4]
{'indices': array([4]), 'celltype': 'not specified'}
Error1! unexpected EOF while parsing (<string>, line 0)
Error1! unexpected EOF while parsing (<string>, line 0)
[{'indices': array([4]), 'celltype': 'not specified'}]
7.433319091796875 	 6.4891815185546875
[19]
{'indices': array([19]), 'celltype': 'not specified'}
Error1! unexpected EOF while parsing (<string>, line 0)
[21]
{'indices': array([21]), 'celltype': 'not specified'}
Error1! unexpected EOF while parsing (<string>, line 0)
Error1! unexpected EOF while parsing (<string>, line 0)
[{'indices': array([19]), 'celltype': 'not specified'}, {'indices': array([21]), 'celltype': 'not specified'}]


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


10.999679565429688 	 14.172744750976562
[2]
{'indices': array([2]), 'celltype': 'not specified'}
Error1! unexpected EOF while parsing (<string>, line 0)
[4]
{'indices': array([4]), 'celltype': 'not specified'}
Error1! unexpected EOF while parsing (<string>, line 0)
[0]
{'indices': array([0]), 'celltype': 'not specified'}
Error1! unexpected EOF while parsing (<string>, line 0)
Error1! unexpected EOF while parsing (<string>, line 0)
[{'indices': array([2]), 'celltype': 'not specified'}, {'indices': array([4]), 'celltype': 'not specified'}, {'indices': array([0]), 'celltype': 'not specified'}]


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:270: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.asanyarray(ar)


8.932876586914062 	 8.571624755859375
[3]
{'indices': array([3]), 'celltype': 'not specified'}
Error1! unexpected EOF while parsing (<string>, line 0)
Error1! unexpected EOF while parsing (<string>, line 0)
[{'indices': array([3]), 'celltype': 'not specified'}]
9.889984130859375 	 9.235000610351562
[18]
{'indices': array([18]), 'celltype': 'not specified'}
Error1! unexpected EOF while parsing (<string>, line 0)
Error1! unexpected EOF while parsing (<string>, line 0)
[{'indices': array([18]), 'celltype': 'not specified'}]
9.132003784179688 	 12.044525146484375
[5]
{'indices': array([5]), 'celltype': 'not specified'}
Error1! unexpected EOF while parsing (<string>, line 0)
Error1! unexpected EOF while parsing (<string>, line 0)
[{'indices': array([5]), 'celltype': 'not specified'}]
11.3494873046875 	 13.818359375


In [21]:
# Generation of the output data

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)',text)]

def get_files(path, file, printnames = False):

    listOfFiles = os.listdir(path)
    pattern = file + '_field*'
    
    i = 0
    filenames = []
    for entry in listOfFiles:
        if fnmatch.fnmatch(entry, pattern):
            filenames.append(entry)
            
    filenames.sort(key = natural_keys)

    return filenames


def get_files_ND2(path, printnames = False):

    listOfFiles = os.listdir(path)
    filenames = []
    for entry in listOfFiles:
        if entry.endswith('nd2'):
            filenames.append(entry)
            
    filenames.sort(key = natural_keys)

    return filenames



def all_names(folder, type, N_patients):
    for i in range(N_patients):
        Leuko_names = []
        path_tmp = folder + '/' + type
        entry_list = get_files(folder, type)
        for j in range(len(entry_list)):
            data_names = pd.read_excel(folder + '/' + entry_list[j],
                                      sheet_name='Mean values')
            #print(data_names.columns)
            headers = list(data_names.columns)
            
            for k in range(len(headers)):
                if 'Type №' in headers[k]:
                    Leuko_names.append(data_names[headers[k]][0])
            print(Leuko_names)     
        Leuko_names.append('all_names')   
        return np.unique(Leuko_names)        
            

def traj(folder, type, N_patients, name_curr):
    for i in range(N_patients):
        instant_velocities = []
        path_tmp = folder + '/' + type

#         path_tmp = folder + '/' + str(i + 1) + 'D' + '/' + type
        entry_list = get_files(folder, type)
#         print(entry_list)
        traj_lens = []
        for j in range(len(entry_list)):
            data_traj = pd.read_excel(folder + '/' + entry_list[j],
                                      sheet_name='Coords')
            headers = list(data_traj.columns)
            X_names = []
            Y_names = []
            for k in range(len(headers)):
                if not (name_curr=='all_names'):
                   try: 
                    if (('Coords X' in headers[k])and(name_curr in data_traj[headers[k+3]][0])):
                        X_names.append(headers[k])
                        #print('trytraj',data_traj[headers[k+3]][0])
                    if (('Coords Y' in headers[k])and(name_curr in data_traj[headers[k+2]][0])):
                        #print('trytraj1',data_traj[headers[k+2]][0])
                        Y_names.append(headers[k])
                   except Exception as e:
                    print(e)
                else:
                    if (('Coords X' in headers[k])):
                        X_names.append(headers[k])
                        #print('trytraj',data_traj[headers[k+3]][0])
                    if (('Coords Y' in headers[k])):
                        #print('trytraj1',data_traj[headers[k+2]][0])
                        Y_names.append(headers[k])

            # print(Y_names)
            for k in range(len(X_names)):
                traj_lens_tmp = []
                X_list = data_traj[X_names[k]].tolist()
                X_list = [x for x in X_list if str(x) != 'nan']
                Y_list = data_traj[Y_names[k]].tolist()
                Y_list = [x for x in Y_list if str(x) != 'nan']
                for k_int in range(1, len(X_list)):
                    dist = np.sqrt((X_list[k_int] - X_list[k_int - 1])**2 +
                                   (Y_list[k_int] - Y_list[k_int - 1])**2)
                    # dist = np.sqrt((data_traj[X_names[k]][l + 1] - data_traj[X_names[k]][l]) ** 2 +
                    #                (data_traj[Y_names[k]][l + 1] - data_traj[Y_names[k]][l]) ** 2)

                    traj_lens_tmp.append(dist)
                traj_lens.append(np.sum(traj_lens_tmp))

    return traj_lens
all_leuco_names = all_names(folder, filename, 1) 

print(all_leuco_names)
dict_trajs = {}
dict_velocities = {}
for unique_names in all_leuco_names: 
    dict_trajs[unique_names] = []
    dict_velocities[unique_names] = []
for unique_names in all_leuco_names:    
    trajectories = traj(folder, filename, 1, unique_names) # Траектории всех нейтрофилов, которые были на наших полях
    print('Traj',trajectories)
    for i in trajectories:
        dict_trajs[unique_names].append(i)        
raw_files = get_files(folder, filename)
import re

thrombus_area_300 = []
thrombus_area_600 = []
velocities = []

for i in range (len(raw_files)):
    pd_import = pd.read_excel(folder + '/' + raw_files[i], sheet_name='Mean values')

    thrombus_area_300.append(pd_import['Unnamed: 0'][1] * 100)
    thrombus_area_600.append(pd_import['Unnamed: 1'][1] * 100)
    columns = list(pd_import.columns)
    for unique_names in all_leuco_names: 
    
    #     print(raw_files[i])
    
        

        
    #     print(columns)
        headers = list(pd_import.columns)
        #print(unique_names)
        for k in range(len(headers)-1):
            print(pd_import[headers[k+1]][0])
            try:
                if not (unique_names=='all_names'):
                    if ((re.search('eloc',headers[k]))and(re.search(unique_names,pd_import[headers[k+1]][0]))):
                        #print('success', unique_names)
                        dict_velocities[unique_names].append(pd_import[headers[k]][0])
                else:
                    print(headers[k])
                    if ((re.search('eloc',headers[k]))):
                        #print('success', unique_names)
                        dict_velocities[unique_names].append(pd_import[headers[k]][0])
            except:
                pass
        #print(dict_velocities[unique_names])
        dict_velocities[unique_names] = [x for x in dict_velocities[unique_names] if str(x) != 'nan']


        velocities_fast = []
        trajectories_fast = []
        for i in range(len(dict_velocities[unique_names])):

            if dict_velocities[unique_names][i] > 0.045:
                velocities_fast.append(dict_velocities[unique_names][i])
                trajectories_fast.append(dict_trajs[unique_names][i])

        out_list = [thrombus_area_300, thrombus_area_600, dict_velocities[unique_names], dict_trajs[unique_names], velocities_fast, trajectories_fast]


        data_out = pd.DataFrame.from_records(out_list).transpose()

        data_out.columns = ['300 s', '600 s', 'Velocities', 'Trajectories', 'Velocities > 0.045', 'Trajectories > 0.045']
        #print(data_out)
        data_out.to_excel(folder + '/' + filename + str(unique_names)+ '_fin.xlsx', index = None)

['not specified', 'not specified']
['not specified', 'not specified']
['not specified', 'not specified', 'not specified']
['not specified', 'not specified', 'not specified', 'not specified', 'not specified']
['not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified']
['not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified']
['not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified']
['not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified']
['not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not specified', 'not 

### all_leuco_names